In [1]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [2]:
%%px
import gzip
import xml.etree.ElementTree as ET
import untangle
import codecs

In [3]:
import os 
path = os.environ.get('ARXIV_META')
#Set path directly when running this file as notebook 
if not path:
    path = os.environ.get('HOME')+'/arxiv/'
print "Read XMLs from", path
zipped_xmls = !ls {path+'*.xml.gz'}
id=0

Read XMLs from /home/SRC_snu/arxiv/


In [4]:
dview.scatter('zipped_xmls', zipped_xmls)
dview.scatter('id', c.ids)
dview['path']=path
%px id=id[0]
#%px id, len(zipped_xmls)

In [5]:
%%px
def GetTitle(record):
    return record.metadata.oai_dc_dc.dc_title.cdata

def GetAbstract(record):
    x = record.metadata.oai_dc_dc.dc_description
    if type(x) is list :
        return x[0].cdata
    return x.cdata

def GetSubjects(record):
    x = record.metadata.oai_dc_dc.dc_subject
    if type(x) is list:
        subjects =  [elm.cdata for elm in x]
    else:
        subjects = [x.cdata]
    #return [x for x in itertools.chain.from_iterable(subjects) if x[0].isalpha()]
    #return [x for x in subjects if x[:2].isalpha()]
    return subjects

def GetHighLevelSubject(subject):
    return subject.split('-')[0]

def YieldAbstracts(xmls):
    for xml in xmls:
        for record in xml.Response.ListRecords.record:
            try:
                yield GetAbstract(record).strip().replace('\n',' ')
            except:
                #print type(record)
                #print record.metadata
                pass

In [6]:
%%px
xml_contents = [gzip.open(zxml).read() for zxml in zipped_xmls]
xmls = [untangle.parse(xml) for xml in xml_contents]

In [ ]:
#For storing abstracts in multiple files
#%%px
#with codecs.open(path+'abstract.%d'%id,'w', 'utf-8') as out:
#    for abstract in YieldAbstracts(xmls):
#        out.write(abstract+'\n')            

In [14]:
import codecs
abstracts = dview.gather('abstracts')
with codecs.open(path+'abstracts','w', 'utf-8') as out:
    for abstract in abstracts.result():
        out.write(abstract+'\n')            